# Technical Notebook 4 - LogReg

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm
%matplotlib inline

In [2]:
# read in CSV saved after cleaning in technical notebook 2 
df = pd.read_csv('final_data/contr-income.csv', 
                 index_col=0
                )
df.shape

(11502, 8)

## Label encoding

In [3]:
# encode the candidates as specific labels 
df['cand_num'] = df.cand_nm
y = df.cand_nm
le = LabelEncoder()
le.fit(y)
df.cand_num = le.transform(y)

In [4]:
print(df.cand_nm.value_counts())
print(df.cand_num.value_counts())

Buttigieg, Pete       3746
Warren, Elizabeth     3609
Sanders, Bernard      2726
Biden, Joseph R Jr    1421
Name: cand_nm, dtype: int64
1    3746
3    3609
2    2726
0    1421
Name: cand_num, dtype: int64


## Split dataset into train and test

In [5]:
df.contbr_zip = df.contbr_zip.astype(str)
df.dtypes

cand_nm               object
contbr_zip            object
contbr_occupation     object
contb_receipt_amt    float64
occ_cat               object
converted_date         int64
zip                  float64
income               float64
cand_num               int64
dtype: object

In [6]:
# zip as dummy variables  
x_feats = ['converted_date',
           'contbr_zip',
           'contb_receipt_amt']
X = pd.get_dummies(df[x_feats],
                   drop_first=True)
X.head()

,converted_date,contb_receipt_amt,contbr_zip_20002,contbr_zip_20003,contbr_zip_20004,contbr_zip_20005,contbr_zip_20006,contbr_zip_20007,contbr_zip_20008,contbr_zip_20009,...,contbr_zip_20015,contbr_zip_20016,contbr_zip_20017,contbr_zip_20018,contbr_zip_20019,contbr_zip_20020,contbr_zip_20024,contbr_zip_20032,contbr_zip_20036,contbr_zip_20037
0,201906,100.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,201906,3.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,201906,27.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,201903,3.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,201906,27.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# target variable is candidate's numeric value 
y = df.cand_num

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    random_state=0, 
                                                    stratify=y, 
                                                    test_size=0.2
                                                   )

## Scale x values 

In [9]:
scaler = StandardScaler().fit(X_train)

X_scaled_train = scaler.transform(X_train)

## Call logistic regression function

In [10]:
clf = LogisticRegression(  C=2, 
                           solver='lbfgs', 
                           multi_class='multinomial', 
                           max_iter=1000, 
                           penalty='l2',
                           class_weight={0: .89 ,1: .78, 2: .98, 3: .87})

## Fit the model

In [11]:
clf = clf.fit(X_scaled_train, y_train)
y_hat_train = clf.predict(X_scaled_train)
print(clf.score(X_scaled_train, y_train))

0.44027823062710575


In [12]:
# create confusion matrix
cm = confusion_matrix(y_train, y_hat_train)
conf_matrix = pd.DataFrame(cm, index=['actual Biden',
                                      'actual Buttigieg',
                                      'actual Sanders',
                                      'actual Warren'], 
                           columns= ['Biden', 
                                     'Buttigieg',
                                     'Sanders',
                                     'Warren'])
conf_matrix

,Biden,Buttigieg,Sanders,Warren
actual Biden,120,435,293,289
actual Buttigieg,86,1197,539,1174
actual Sanders,18,241,992,930
actual Warren,31,488,626,1742


In [13]:
# cross validate train,  k = 3 
clf_cv = cross_val_score(clf,
                        X_scaled_train, 
                        y_train, 
                        cv=3, 
                        )
clf_cv

array([0.44426336, 0.4189762 , 0.42889759])

In [14]:
X_test = scaler.transform(X_test)
y_hat_test = clf.predict(X_test)
cm = confusion_matrix(y_train, y_hat_train)
conf_matrix = pd.DataFrame(cm, index=['actual Biden',
                                      'actual Buttigieg',
                                      'actual Sanders',
                                      'actual Warren'], 
                           columns= ['Biden', 
                                     'Buttigieg',
                                     'Sanders',
                                     'Warren'])
conf_matrix

,Biden,Buttigieg,Sanders,Warren
actual Biden,120,435,293,289
actual Buttigieg,86,1197,539,1174
actual Sanders,18,241,992,930
actual Warren,31,488,626,1742


In [15]:
print(clf.score(X_test, y_test))

0.4211212516297262


## Optimize for Sanders metric
Classify correctly as many Sanders contributors as possible. 